In [ ]:
df = pd.read_csv("archive/train.csv" )

# Map target to binary
if "churn" not in df.columns:
    raise ValueError("Expected a 'churn' column in the training CSV.")
df["churn_binary"] = df["churn"].map({"yes": 1, "no": 0})

X = df.drop(columns=["churn", "churn_binary"])
y = df["churn_binary"]

display(df.head())
print("Shape:", df.shape)
print("Target positive rate:", y.mean().round(4))


In [2]:
#train split 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

cat_cols = X.select_dtypes(include=["object"]).columns.tolist()
num_cols = X.select_dtypes(exclude=["object"]).columns.tolist()

print("Categorical:", cat_cols)
print("Numerical:", num_cols)

In [ ]:
#preprocessing
numeric_transformer = Pipeline([("scaler", StandardScaler())])
categorical_transformer = Pipeline([("ohe", OneHotEncoder(handle_unknown="ignore"))])

preprocess = ColumnTransformer(
    [("num", numeric_transformer, num_cols),
     ("cat", categorical_transformer, cat_cols)]
)


In [ ]:
#training
pipe_lr = Pipeline([
    ("preprocess", preprocess),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

pipe_rf = Pipeline([
    ("preprocess", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=300, class_weight="balanced", random_state=42
    ))
])

pipe_lr.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)
print("Models trained.")


In [ ]:
#generate best model
proba_lr = pipe_lr.predict_proba(X_test)[:, 1]
proba_rf = pipe_rf.predict_proba(X_test)[:, 1]

auc_lr = roc_auc_score(y_test, proba_lr)
auc_rf = roc_auc_score(y_test, proba_rf)

if auc_rf >= auc_lr:
    best_name, best_model, y_proba = "RandomForest", pipe_rf, proba_rf
else:
    best_name, best_model, y_proba = "LogisticRegression", pipe_lr, proba_lr

print(f"Best model: {best_name} | AUC_LR={auc_lr:.3f} | AUC_RF={auc_rf:.3f}")


In [ ]:
#evaluation 0.5
y_pred = (y_proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
roc_auc = roc_auc_score(y_test, y_proba)
avg_prec = average_precision_score(y_test, y_proba)

print(f"Accuracy={acc:.3f}, Precision={prec:.3f}, Recall={rec:.3f}, "
      f"F1={f1:.3f}, ROC-AUC={roc_auc:.3f}, PR-AUC(AP)={avg_prec:.3f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
#generate graphs
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure(figsize=(6, 4))
plt.plot(fpr, tpr, label=f"{best_name} (AUC={roc_auc:.3f})")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve"); plt.legend(loc="lower right"); plt.tight_layout()
plt.show()

pr_prec, pr_rec, _ = precision_recall_curve(y_test, y_proba)
plt.figure(figsize=(6, 4))
plt.plot(pr_rec, pr_prec, label=f"{best_name} (AP={avg_prec:.3f})")
plt.xlabel("Recall"); plt.ylabel("Precision")
plt.title("Precision-Recall Curve"); plt.legend(loc="lower left"); plt.tight_layout()
plt.show()


In [ ]:
#save model
MODEL_PATH = "churn_model_pipeline.joblib"
import joblib
joblib.dump(best_model, MODEL_PATH)
print(f"Model saved -> {MODEL_PATH}")

In [ ]:
#try model on test.csv
NEW_DATA_CSV = "archive/test.csv"
new_data = pd.read_csv(NEW_DATA_CSV)
proba_new = best_model.predict_proba(new_data)[:, 1]
pred_new = (proba_new >= 0.5).astype(int)
out = new_data.copy()
out["churn_proba"] = proba_new
out["churn_pred"] = pred_new
out.to_csv("predictions.csv", index=False)
print("Predictions saved to predictions.csv")

In [ ]:
#fine tune
thresholds = np.linspace(0.1, 0.9, 17)
best_thr, best_f1 = 0.5, -1.0
for thr in thresholds:
    yp = (y_proba >= thr).astype(int)
    f1_thr = f1_score(y_test, yp, zero_division=0)
    if f1_thr > best_f1:
        best_f1, best_thr = f1_thr, thr
print(f"Best F1={best_f1:.3f} at threshold={best_thr:.2f}")


In [ ]:
#generate importance graph
try:
    pipe = best_model
except NameError:
    pipe = joblib.load("churn_model_pipeline.joblib")

# post-preprocessing feature names
pre = pipe.named_steps["preprocess"]
num_cols, cat_cols = [], []
for name, transformer, cols in pre.transformers_:
    if name == "num":
        num_cols = list(cols)
    elif name == "cat":
        cat_cols = list(cols)

ohe = pre.named_transformers_["cat"].named_steps["ohe"]
try:
    cat_feature_names = ohe.get_feature_names_out(cat_cols).tolist()
except Exception:
    cat_feature_names = list(ohe.get_feature_names(cat_cols))

feature_names = list(num_cols) + cat_feature_names

# importances
clf = pipe.named_steps["clf"]
metric_name = "importance"

if isinstance(clf, RandomForestClassifier):
    values = clf.feature_importances_
elif isinstance(clf, LogisticRegression):
    values = np.abs(clf.coef_.ravel())  # magnitude as importance
    metric_name = "abs_coef"
else:
    # Fallback: permutation importance (uses X_test/y_test from earlier cells)
    result = permutation_importance(pipe, X_test, y_test, n_repeats=5, random_state=42, n_jobs=1)
    values = result.importances_mean
    metric_name = "perm_importance"

feat_df = pd.DataFrame({"feature": feature_names, metric_name: values}).sort_values(metric_name, ascending=False).reset_index(drop=True)

# Top 20
topn = 20
top_df = feat_df.head(topn).iloc[::-1]  # reverse for barh
plt.figure(figsize=(10, 8))
plt.barh(top_df["feature"], top_df[metric_name])
plt.xlabel("Importance")
plt.title("Top Feature Importances")
plt.tight_layout()
plt.show()

# Save figure
plt.figure(figsize=(10, 8))
plt.barh(top_df["feature"], top_df[metric_name])
plt.xlabel("Importance")
plt.title("Top Feature Importances")
plt.tight_layout()
plt.savefig("feature_importances_top20.png", dpi=180)
plt.close()

# Preview
feat_df.head(50)

In [ ]:
#Plot Generation
df = pd.read_csv("archive/train.csv")


# Data cleaning, string to int
df["churn"] = df["churn"].str.strip().str.lower().map({"yes": 1, "no": 0})

y = df["churn"].astype(int)
X = df.drop(columns=["churn"])

# Restrict to most impactful features
selected_features = [
    "number_customer_service_calls",
    "total_intl_minutes",
    "total_day_minutes",
    "total_day_charge"
]
X = X[selected_features]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


models = {
    "Logistic Regression": LogisticRegression(max_iter=500, random_state=42),
    "Decision Tree": DecisionTreeClassifier(max_depth=6, random_state=42),
    "Random Forest": RandomForestClassifier(
        n_estimators=200, max_depth=8, random_state=42
    ),
}


for name, model in models.items():
    print(f"\n===== {name} =====")

    
    if name == "Logistic Regression":
        model.fit(X_train_scaled, y_train)
        y_probs = model.predict_proba(X_test_scaled)[:, 1]
        y_pred = model.predict(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_probs = model.predict_proba(X_test)[:, 1]
        y_pred = model.predict(X_test)

    
    roc_auc = roc_auc_score(y_test, y_probs)
    ap_score = average_precision_score(y_test, y_probs)

    print(f"ROC-AUC: {roc_auc:.3f} | PR-AUC(AP): {ap_score:.3f}")
    print(classification_report(y_test, y_pred, digits=3))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Reds")
    plt.title(f"Confusion Matrix — {name}")
    plt.xlabel("Predicted"); plt.ylabel("True")
    plt.tight_layout()
    plt.show()

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    plt.figure(figsize=(5, 4))
    plt.plot(fpr, tpr, label=f"{name} (AUC={roc_auc:.3f})")
    plt.plot([0, 1], [0, 1], "--", color="gray")
    plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve — {name}")
    plt.legend(); plt.tight_layout()
    plt.show()

    # Precision–Recall Curve
    prec, rec, _ = precision_recall_curve(y_test, y_probs)
    plt.figure(figsize=(5, 4))
    plt.plot(rec, prec, label=f"{name} (AP={ap_score:.3f})")
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(f"Precision–Recall Curve — {name}")
    plt.legend(); plt.tight_layout()
    plt.show()

model = sm.OLS(y, X).fit()

print(model.summary())